In [ ]:
import os
import shutil
import random
import time
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

# === RANDOM SEED (for reproducibility) ===
random.seed(42)
np.random.seed(42)

# === USER PATHS ===
source_path = r"C:\Users\moham\OneDrive\Desktop\Project-1\Data\train"
target_path = r"C:\Users\moham\OneDrive\Desktop\Project-1\Data_augmented\train"

# === CREATE target folder ===
os.makedirs(target_path, exist_ok=True)

# === IMAGE SIZE (make all images consistent) ===
IMAGE_SIZE = (48, 48)  # change if your model expects different size (e.g. (64,64))

# === AUGMENTATION SETTINGS (enhanced) ===
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),      # NEW → random brightness variation
    fill_mode='nearest'
)

TARGET_COUNT = 10000  # final images per class

# Try to import tqdm for a progress bar
try:
    from tqdm import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False


# === START AUGMENTATION PROCESS ===
for class_name in sorted(os.listdir(source_path)):
    class_src = os.path.join(source_path, class_name)
    if not os.path.isdir(class_src):
        continue

    class_dst = os.path.join(target_path, class_name)
    os.makedirs(class_dst, exist_ok=True)

    # 1) Copy original images
    existing_images = [f for f in os.listdir(class_src) if os.path.isfile(os.path.join(class_src, f))]
    if len(existing_images) == 0:
        print(f"[{class_name}] ⚠️ No images found. Skipping.")
        continue

    for file in existing_images:
        src_file = os.path.join(class_src, file)
        dst_file = os.path.join(class_dst, file)
        if not os.path.exists(dst_file):
            shutil.copy(src_file, dst_file)

    usage_counts = {fname: 0 for fname in existing_images}

    current_count = len(os.listdir(class_dst))
    print(f"[{class_name}] Starting with {current_count} originals...")

    remaining = max(0, TARGET_COUNT - current_count)
    if remaining == 0:
        print(f"[{class_name}] Already has {TARGET_COUNT}. Skipping.")
        continue

    if use_tqdm:
        pbar = tqdm(total=remaining, desc=f"Augmenting {class_name}", unit="img")
    else:
        last_print_time = time.time()

    # 2) Augmentation loop
    while current_count < TARGET_COUNT:
        min_usage = min(usage_counts.values())
        pool = [fname for fname, cnt in usage_counts.items() if cnt == min_usage]
        chosen = random.choice(pool)

        img_path = os.path.join(class_src, chosen)
        try:
            img = load_img(img_path, target_size=IMAGE_SIZE)
        except Exception as e:
            print(f"[{class_name}] ❌ Could not load {chosen}: {e}. Skipping.")
            usage_counts.pop(chosen, None)
            if len(usage_counts) == 0:
                raise RuntimeError(f"[{class_name}] No usable images left!")
            continue

        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        for batch in datagen.flow(x, batch_size=1):
            current_count += 1
            usage_counts[chosen] += 1
            filename = f"{class_name}_{current_count:05d}.jpg"
            save_path = os.path.join(class_dst, filename)
            array_to_img(batch[0]).save(save_path)
            break  # one augmented image per iteration

        if use_tqdm:
            pbar.update(1)
        else:
            if (current_count % 200 == 0) or (time.time() - last_print_time > 10):
                print(f"[{class_name}] {current_count}/{TARGET_COUNT} done...")
                last_print_time = time.time()

    if use_tqdm:
        pbar.close()

    print(f"[{class_name}] ✅ DONE → {current_count} images (max usage: {max(usage_counts.values())})")

print("\n✅ ALL CLASSES successfully augmented to", TARGET_COUNT, "images each.")
print("📂 Saved in:", target_path)


[angry] Starting with 4102 originals in augmented folder...


Augmenting angry: 100%|██████████| 5898/5898 [00:27<00:00, 212.45img/s]


[angry] ✅ DONE → 10000 images total. (max usage per original: 2)
[disgust] Starting with 613 originals in augmented folder...


Augmenting disgust: 100%|██████████| 9387/9387 [00:37<00:00, 251.48img/s]


[disgust] ✅ DONE → 10000 images total. (max usage per original: 16)
[fear] Starting with 4172 originals in augmented folder...


...

[neutral] ✅ DONE → 10000 images total. (max usage per original: 1)
[sad] Starting with 4921 originals in augmented folder...


Augmenting sad: 100%|██████████| 5079/5079 [00:24<00:00, 206.81img/s]


[sad] ✅ DONE → 10000 images total. (max usage per original: 2)
[surprise] Starting with 3417 originals in augmented folder...


Augmenting surprise: 100%|██████████| 6583/6583 [00:35<00:00, 183.88img/s]

[surprise] ✅ DONE → 10000 images total. (max usage per original: 2)

✅ ALL CLASSES successfully augmented to 10000 images EACH.
📂 Saved in: C:\Users\moham\OneDrive\Desktop\Project-1\Data_augmented\train
